
# IBM Applied Data Science Capstone, Final Report (Week 4 and 5)

***
## Background

The client wants to build and run an Indonesian restaurant in Stockholm, Sweden. He wants to build the restaurant close to the body of water, as he believes it would fulfill his childhood dream. Although he lived long enough already in Stockholm, he is still unaware of the distribution of other similar type of restaurants as they would be potential competitors. He is very well funded and connected to the culinary business in Stockholm, so that supply distribution wouldn't be a major problem.

***
## Objectives
- Find a list of neighborhoods in Stockholm city.
- Find the intensity of certain venues in each neighborhood that potentially become competitors to the client's restaurant.
- Using k-clustering, find the neighborhood where potentially competitor venues has the least value, as this would show some potential site for the restaurant.
- Cross-correlate the potential site with the surrounding body of water(river, lake, sea).
- Find a neighborhood or two in Stockholm that is the most suitable to build the restaurant. Recommend the neighborhood to client with its pros and cons.

***
## Data Requirements
To achieve the goal of this project, we need the data in forms of a list of neighborhood on Stockholm city, Sweden. The neighborhood in Sweden is available to the boroughs(which consist of multiple districts) and districts, where the latter are more suitable in this context. The list of districts can be obtained via Wikipedia. Other data that we need is a map of Stockholm, which shows main streets and land allocations.  To assign each neighborhood automatically into the map, we need latitude and longitude coordinate data from Geocoder library.

***
## Methodology and Results
### Import libraries

In [43]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [44]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [45]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [59]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### Scrape data from suitable wikipedia article that contains a list of districts in Stockholm, Sweden. Convert the data into a dataframe.

In [60]:
# send the GET request
sto_get = requests.get("https://en.wikipedia.org/wiki/Category:Districts_of_Stockholm").text

soup_data = BeautifulSoup(sto_get, 'html.parser')

neighborhoodList = []
for row in soup_data.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

# create a new DataFrame from the list
sto_df = pd.DataFrame({"Neighborhood": neighborhoodList})

sto_df.head(25)

,Neighborhood
0,Abrahamsberg
1,Alvik
2,Älvsjö
3,Aspudden
4,Axelsberg
5,Bagarmossen
6,Bandhagen
7,Birkastaden
8,Björkhagen
9,Blackeberg


In [61]:
sto_df.shape

(68, 1)

### Obtain geographical coordinates (latitude,longitude) from each neighborhood in Stockholm

In [62]:
# a function to retrieve coordinate data from each neighborhood
def get_lat_lng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Stockholm, Sweden'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [63]:
# call the function to get the coordinates, store in a new list using list comprehension
sto_coords = [ get_lat_lng(neighborhood) for neighborhood in sto_df["Neighborhood"].tolist() ]

# create temporary dataframe to populate the coordinates into Latitude and Longitude
sto_coords_df = pd.DataFrame(sto_coords, columns=['Latitude', 'Longitude'])

# merge the coordinates dataframe and neighborhood dataframe into the main dataframe
sthlm_df = pd.concat([sto_df, sto_coords_df], axis=1) #sto_df['Latitude'] = sto_coords['Latitude']


In [64]:
# save the DataFrame as CSV file
sthlm_df.to_csv("sthlm_df.csv", index=False)

### Build a map of Stockholm city.

In [65]:
# retrieve the coordinates of Stockholm
address = 'Stockholm City, Sweden'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Stockholm using latitude and longitude values
sthlm_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(sthlm_df['Latitude'], sthlm_df['Longitude'], sthlm_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(sthlm_map)  

# save the map as HTML file
sthlm_map.save('sthlm_map.html')

# show the constructed map   
sthlm_map

### Using the Foursquare API,  let's begin to search for venues in each neighborhood.

In [66]:
CLIENT_ID = 'P40F20UHNUGBTM0V0251ETOQYTT442BJEJ3KG42O2QL2DSNV' # your Foursquare ID
CLIENT_SECRET = 'CHV223VTL3F5OCLRHGKTVCF313K1ZBYOSSPKOS0UA5ODBIMR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: P40F20UHNUGBTM0V0251ETOQYTT442BJEJ3KG42O2QL2DSNV
CLIENT_SECRET:CHV223VTL3F5OCLRHGKTVCF313K1ZBYOSSPKOS0UA5ODBIMR


### Retrieve the top 100 venues that are within a radius of 1500 meters.

In [67]:
radius = 1500
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(sthlm_df['Latitude'], sthlm_df['Longitude'], sthlm_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [69]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(10)

(3803, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abrahamsberg,59.33363,17.95072,Friskis & Svettis Abrahamsberg,59.337785,17.948774,Gym / Fitness Center
1,Abrahamsberg,59.33363,17.95072,Gelato Scarfó,59.323969,17.956710,Ice Cream Shop
2,Abrahamsberg,59.33363,17.95072,Stora Mossens Ishall,59.329441,17.961769,Skating Rink
3,Abrahamsberg,59.33363,17.95072,Daisy's,59.340587,17.961140,Fast Food Restaurant
4,Abrahamsberg,59.33363,17.95072,Park konditori,59.329018,17.973576,Bakery
5,Abrahamsberg,59.33363,17.95072,Nockeby Bageri,59.328910,17.927926,Bakery
6,Abrahamsberg,59.33363,17.95072,Stora Mossens IP,59.329345,17.960668,Track
7,Abrahamsberg,59.33363,17.95072,Pizzeria La Bella,59.333929,17.947791,Pizza Place
8,Abrahamsberg,59.33363,17.95072,Elliots,59.327796,17.974019,Restaurant
9,Abrahamsberg,59.33363,17.95072,Ulvsunda Slott,59.340859,17.964221,Castle


 ### Analyze Each Neighborhood using one-hot encoding.

In [71]:
# one hot encoding
sthlm_ohe = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sthlm_ohe['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sthlm_ohe.columns[-1]] + list(sthlm_ohe.columns[:-1])
sthlm_ohe = sthlm_ohe[fixed_columns]

print(sthlm_ohe.shape)
sthlm_ohe.head()

(3803, 242)


,Neighborhoods,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Bathing Area,Bay,Beach,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Camera Store,Campground,Canal,Candy Store,Caribbean Restaurant,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Church,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Doner Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General College & University,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Kurdish Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Mongolian Restaurant,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,Nature Preserve,Neighborhood,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Gym,Outdoor Sculpture,Outlet Store,Palace,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Racetrack,Rafting,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Roof Deck,Rugby Pitch,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse,Wine Bar,Women's Store,Yoga Studio,Zoo Exhibit
0,Abrahamsberg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Abrahamsberg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [72]:
sthlm_groupbynbr = sthlm_ohe.groupby(["Neighborhoods"]).mean().reset_index()


(68, 242)


,Neighborhoods,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Bathing Area,Bay,Beach,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Camera Store,Campground,Canal,Candy Store,Caribbean Restaurant,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Church,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Doner Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General College & University,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Kurdish Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Mongolian Restaurant,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,Nature Preserve,Neighborhood,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Gym,Outdoor Sculpture,Outlet Store,Palace,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Racetrack,Rafting,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Roof Deck,Rugby Pitch,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse,Wine Bar,Women's Store,Yoga Studio,Zoo Exhibit
0,Abrahamsberg,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.085714,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.028571,0.0,0.028571,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.00,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.057143,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.028571,0.0,0.000000,0.028571,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.028571,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

### Search for venues that potentially become competitor for the client. As the client want to build Indonesian restaurant, any other Indonesian Restaurant in Stockholm would be a direct competitor. For now, let's search for Indonesian restaurant around Stockholm, and see how their intensity in each neighborhood.

In [80]:
venue_name = "Indonesian Restaurant"
sthlm_venue = sthlm_groupbynbr[sthlm_groupbynbr[venue_name] > 0]
sthlm_venue_hunt = sthlm_groupbynbr[["Neighborhoods",venue_name]]

In [81]:
sthlm_venue_hunt

,Neighborhoods,Indonesian Restaurant
0,Abrahamsberg,0.000000
1,Alvik,0.000000
2,Aspudden,0.000000
3,Axelsberg,0.000000
4,Bagarmossen,0.000000
5,Bandhagen,0.000000
6,Birkastaden,0.000000
7,Björkhagen,0.000000
8,Blackeberg,0.000000
9,Bredäng,0.000000


### Luckily, there are only few of Indonesian Restaurants in Stockholm. But still, we need to visualize the data into the map so our client can see the distribution of Indonesian restaurants and have a better understanding.


### Let's utilize the k-clustering tool to visualize the intensity distribution.

In [82]:
#### set number of clusters
kclusters = 4

sthlm_kcl = sthlm_venue_hunt.drop(["Neighborhoods"], 1)
sthlm_kcl.head()

,Indonesian Restaurant
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [83]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sthlm_kcl)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
sthlm_merged = sthlm_venue_hunt.copy()

# add clustering labels
sthlm_merged["Cluster Labels"] = kmeans.labels_

sthlm_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
sthlm_merged.head(20)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

### Here, we visualize the constructed k-clustering ( in context of Indonesian restaurant intensity distribution) into the map.

In [88]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sthlm_merged['Latitude'], sthlm_merged['Longitude'], sthlm_merged['Neighborhood'], sthlm_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# save the map as HTML file
map_clusters.save('map_clusters_1.html')

### We see that the distribution of Indonesian restaurants is limited to western central part of Stockholm. Naturally, we will avoid these neighborhood when recommending a place for our client. Alternatively, we can check for the cluster with highest intensity below:

In [90]:
sthlm_merged.loc[sthlm_merged['Cluster Labels'] == 2]

,Neighborhood,Indonesian Restaurant,Cluster Labels,Latitude,Longitude
45,Reimersholme,0.010753,2,59.31785,18.02154


## Beside Indonesian restaurants, our client also perceive general Asian restaurants as potential competitors for his restaurant. Therefore, we need to build a similar clustering in context of Asian restaurants around Stockholm. 

In [91]:
sthlm_groupbynbr = sthlm_ohe.groupby(["Neighborhoods"]).mean().reset_index()

print(sthlm_groupbynbr.shape)
sthlm_groupbynbr.head(5)

(68, 242)


,Neighborhoods,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Bathing Area,Bay,Beach,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Camera Store,Campground,Canal,Candy Store,Caribbean Restaurant,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Church,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Doner Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General College & University,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Kurdish Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Mongolian Restaurant,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,Nature Preserve,Neighborhood,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Gym,Outdoor Sculpture,Outlet Store,Palace,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Racetrack,Rafting,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Roof Deck,Rugby Pitch,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse,Wine Bar,Women's Store,Yoga Studio,Zoo Exhibit
0,Abrahamsberg,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.085714,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.028571,0.0,0.028571,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.00,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.057143,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.028571,0.0,0.000000,0.028571,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.028571,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [92]:
venue_name = "Asian Restaurant"

sthlm_venue = sthlm_groupbynbr[sthlm_groupbynbr[venue_name] > 0]

len(sthlm_venue)

sthlm_venue_hunt = sthlm_groupbynbr[["Neighborhoods",venue_name]]

kclusters = 7

sthlm_kcl = sthlm_venue_hunt.drop(["Neighborhoods"], 1)
sthlm_kcl.head()

,Asian Restaurant
0,0.00
1,0.00
2,0.02
3,0.00
4,0.00


In [93]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sthlm_kcl)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
sthlm_merged = sthlm_venue_hunt.copy()

# add clustering labels
sthlm_merged["Cluster Labels"] = kmeans.labels_

sthlm_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
sthlm_merged.head(20)

,Neighborhood,Asian Restaurant,Cluster Labels
0,Abrahamsberg,0.000000,0
1,Alvik,0.000000,0
2,Aspudden,0.020000,4
3,Axelsberg,0.000000,0
4,Bagarmossen,0.000000,0
5,Bandhagen,0.000000,0
6,Birkastaden,0.010000,5
7,Björkhagen,0.000000,0
8,Blackeberg,0.040000,6
9,Bredäng,0.000000,0


In [94]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sthlm_merged = sthlm_merged.join(sthlm_df.set_index("Neighborhood"), on="Neighborhood")

print(sthlm_merged.shape)
sthlm_merged.head() # check the last columns!

(68, 5)


,Neighborhood,Asian Restaurant,Cluster Labels,Latitude,Longitude
0,Abrahamsberg,0.00,0,59.333630,17.950720
1,Alvik,0.00,0,59.816670,18.900000
2,Aspudden,0.02,4,59.305630,18.002190
3,Axelsberg,0.00,0,59.303318,17.974385
4,Bagarmossen,0.00,0,59.278080,18.128210


In [95]:
# sort the results by Cluster Labels
print(sthlm_merged.shape)
sthlm_merged.sort_values(["Cluster Labels"], inplace=True)
sthlm_merged

(68, 5)


,Neighborhood,Asian Restaurant,Cluster Labels,Latitude,Longitude
0,Abrahamsberg,0.000000,0,59.333630,17.950720
29,Högdalen,0.000000,0,59.264120,18.042880
30,Hökarängen,0.000000,0,59.255830,18.081440
66,Örby slott,0.000000,0,59.282290,18.032030
35,Kärrtorp,0.000000,0,59.284390,18.112230
41,Mälarhöjden,0.000000,0,59.301430,17.951990
42,Nockeby,0.000000,0,59.329420,17.928500
44,Orhem,0.000000,0,59.248030,18.139770
46,Riddarholmen,0.000000,0,59.324780,18.063120
47,Riksby,0.000000,0,59.340420,17.941630


In [96]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, venue_score in zip(sthlm_merged['Latitude'], sthlm_merged['Longitude'], sthlm_merged['Neighborhood'], sthlm_merged['Cluster Labels'],sthlm_merged[venue_name] ):
    label = folium.Popup(str(poi) + '(Cluster ' + str(cluster)+ ') | Score :'+ str(venue_score), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters.save('map_clusters_2.html')

map_clusters

#### The clustering of Asian Restaurants shows significantly different results compared to Indonesian restaurants. Here, we can see that Asian restaurants are more widely available and have higher intensity compared to Indonesian one. We will have harder time to choose the suitable neighborhood for our client.

***

## Discussion

From the result we have multiple sites that we can recommend to the client. However, despite the fact that some neighborhood is close to body of water and has low to no value of Indonesian and Asian Restaurants, some issue can happen that make these neighborhood unsuitable for restaurants/general businesses.
We briefly discuss each potential neighborhood below:

- Areas in easter central Stockholm (except Skeppsholmen) is virtually impossible, since the available land is very limited and is already reserved by government institution/large corporate.

- Skeppsholmen: This is among the best area because it's close to the city center. Expect very expensive property rent/construction.
- Farsta, Farsta Strand and Skondal: These areas are close to body of water and somewhat close to the city center.
- Orhem and Skrubba: These places are not fully developed and difficult to reach for general public. We cannot recommend these areas.
- Mälarhöjden : This area is  close to body of water and somewhat close to the city center via public transport.
- Bredäng: The land beside the body of water in Bredang is reserved for nature reservation.
- Axelberg : Good place, but its neighboring area has a few Asian restaurants.
- Stora Essingen : These areas are close to body of water and somewhat close to the city center.
- Nockeby : These areas are close to body of water, but quite distant to the city center.
- Kungshamra : These areas are close to body of water, but quite distant to the city center.
- Stadshagen :This is among the best area because it's close to the city center. Expect very expensive property rent/construction.

***

## Conclusion

From the result, we conclude that we can only recommend a few neighborhood to our client. Below, we ranked these neighborhoods based on multiple factors, primarily due to competitor presence, surrounding body of water, distance to city's center, and other relevant issues:

1. Skeppsholmen

2. Stadshagen

3. Stora Essingen 

4. Mälarhöjden

5. Farsta, Farsta Strand and Skondal


# THE END